# project_ef21_extension: Script generators 

In [5]:
%matplotlib inline
import numpy as np
from sklearn.model_selection import train_test_split
import time
import sys
import os
import argparse
from numpy.random import normal, uniform

from numpy.linalg import norm
import itertools
from scipy.special import binom
from scipy.stats import ortho_group
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd
from matplotlib import pyplot as plt
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
import datetime
from IPython import display
from logreg_functions_fast import *

from contextlib import redirect_stdout
import shutil
import subprocess

In [9]:
%%bash
python3 generate_data.py --dataset realsim --num_starts 1 --num_workers 100 --loss_func log-reg

(72309,) [-1. -1. -1. -1. -1.]
Data shape:  (72309, 20958)
worker 0 has 723 datasamples; class 1: 723; class -1: 0
worker 1 has 723 datasamples; class 1: 723; class -1: 0
worker 2 has 723 datasamples; class 1: 723; class -1: 0
worker 3 has 723 datasamples; class 1: 723; class -1: 0
worker 4 has 723 datasamples; class 1: 723; class -1: 0
worker 5 has 723 datasamples; class 1: 723; class -1: 0
worker 6 has 723 datasamples; class 1: 723; class -1: 0
worker 7 has 723 datasamples; class 1: 723; class -1: 0
worker 8 has 723 datasamples; class 1: 723; class -1: 0
worker 9 has 723 datasamples; class 1: 723; class -1: 0
worker 10 has 723 datasamples; class 1: 723; class -1: 0
worker 11 has 723 datasamples; class 1: 723; class -1: 0
worker 12 has 723 datasamples; class 1: 723; class -1: 0
worker 13 has 723 datasamples; class 1: 723; class -1: 0
worker 14 has 723 datasamples; class 1: 723; class -1: 0
worker 15 has 723 datasamples; class 1: 723; class -1: 0
worker 16 has 723 datasamples; class 1:

## Bash script generator

# Full gradient case for generaly non-convex functions  with theoretical stepsize

## fatkhui

In [5]:
myrepr = lambda x: repr(round(x, 4)).replace('.',',') if isinstance(x, float) else repr(x)

def read_file (shell_script_name, print_each_script):
    if print_each_script:
        f = open(shell_script_name, 'r')
        print(f.read())
        f.close()

project_path = os.getcwd() + "/"

dataset_ar = ["mushrooms","w8a","a9a","phishing"]
#dataset_ar = ["mushrooms","a9a","phishing"]
#dataset_ar = ["mushrooms"]
#dataset_ar = ["w8a"]
#dataset_ar = ["a9a"]
#dataset_ar = ["phishing"]

exps = ["ef21_fg"]
exps = ["ef21_mnt"]
exps = ["ef21_pp"]
#exps = ["ef21_sgd"]
#exps = ["ef21_hb"]
#exps = ["ef21_plus_fg"]
#exps = ["ef21_page"]
#exps = ["hb"]
#exps = ["gd"]
#exps = ["ef21_bc"]

tol = 1e-7

############################
##EF21 with full gradients##
############################
#n=20
ef21_fg_mushrooms_num_iter = 2355962
ef21_fg_w8a_num_iter = 155693
ef21_fg_a9a_num_iter = 56249
ef21_fg_phishing_num_iter = 4549

ef21_fg_k_ar = [1,2,4,8,16,32,64]
ef21_fg_factor_ar = [1,2,4,8,16,32,64,128,256,512,1024,2048,4096]

##################
## EF21 with HB ##
##################

ef21_hb_mushrooms_num_iter = 100_000
ef21_hb_w8a_num_iter = 100000
ef21_hb_a9a_num_iter = 10000
ef21_hb_phishing_num_iter = 10000

ef21_hb_k_ar = [1,2,4,8,16]
#ef21_hb_k_ar = [1]

ef21_hb_factor_ar = [1,2,4,8,16,32,64,128,256,512,1024,2048]
ef21_hb_factor_ar = [0.125,0.25,0.5]
ef21_hb_factor_ar = [0.25]
#ef21_hb_factor_ar = [64,128,256,512,1024,2048,4096]
ef21_hb_eta_ar = np.array([0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,0.99]) # final set for all 
#ef21_hb_eta_ar = np.sort( np.concatenate ((np.linspace(start=0.01, stop=0.99,endpoint=True,num=20),np.array([0.9,0.95])))) #for a9a

####################
## EF21 with PAGE ##
####################
ef21_page_mushrooms_num_iter = 23_000
ef21_page_w8a_num_iter = 29_000
ef21_page_a9a_num_iter = 5_000
ef21_page_phishing_num_iter = 5_000

ef21_page_k_ar = [1,2,4,8,16]
ef21_page_factor_ar = [0.25,0.5,1,2,4,8,16,32,64,128,256,512,1024,2048]
ef21_page_prb_ar = [0.95,0.5,0.25]

##################
## EF21 with PP ##
##################
ef21_pp_mushrooms_num_iter = 30_000
ef21_pp_w8a_num_iter = 100_000
ef21_pp_a9a_num_iter = 100_000
ef21_pp_phishing_num_iter = 100_000

ef21_pp_k_ar = [2]
ef21_pp_factor_ar = np.array([0.25,0.5,1,2,4,8,16,32,64,128,256],dtype=float)
ef21_pp_factor_ar = np.array([512,1024,2048,4096,9192],dtype=float)

dependence_steps_k = 0

ef21_pp_prw_ar = [0.95,0.5,0.25,0.125]

###################
## EF21 with SGD ##
###################
ef21_sgd_mushrooms_num_iter = 23_000
ef21_sgd_w8a_num_iter = 29_000
ef21_sgd_a9a_num_iter = 5_000
ef21_sgd_phishing_num_iter = 5_000

ef21_sgd_k_ar = [1,2,4,8,16]
ef21_sgd_factor_ar = [0.25,0.5,1,2,4,8,16,32,64,128,256,512,1024,2048]
ef21_sgd_prb_ar = [0.95,0.5,0.25,0.125,0.065,0.03]

########
## HB ##
########
hb_mushrooms_num_iter = 1_000
hb_w8a_num_iter = 10_000
hb_a9a_num_iter = 5_000
hb_phishing_num_iter = 5_000

hb_factor_ar = [0.125,0.25,0.5, 1,2,4,8,16,32,64,128,256,512,1024,2048]
hb_factor_ar = [4096,8192]
#hb_factor_ar = [0.25]
hb_eta_ar = np.array([0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,0.99]) # final set for all 

########
## GD ##
########
gd_mushrooms_num_iter = 1_000
gd_w8a_num_iter = 1_000
gd_a9a_num_iter = 1_000
gd_phishing_num_iter = 1_000
gd_factor_ar = [1,2,4,8,16,32,64,128,256,512,1024,2048]
gd_factor_ar = [0.125,0.25,0.5]

################
##EF21 with BC##
################
ef21_bc_mushrooms_num_iter = 5_000
ef21_bc_w8a_num_iter = 5_000
ef21_bc_a9a_num_iter = 1_000
ef21_bc_phishing_num_iter = 1000

ef21_bc_k_od_ar = [1,2,4,8,16] #ef21 top-k
ef21_bc_k_bd_ar = [1,2,4,8,16]
ef21_bc_k_od_ar = [1,2]
ef21_bc_k_bd_ar = [32,64]
ef21_bc_factor_ar = [1,2,4,8,16,32,64,128,256,512,1024,2048,4096,9192]

if "ef21_pp" in exps:
    n_ar = [100]
else:
    n_ar = [20]

time = 4
cpus_per_task = 1
memmory = 8

print_each_script = 1
print_global_script = 1

shell_script_name = "slurm_ef21_hb_k1-16_f64"
shell_script_name = "slurm_ef21_page_k1-16_f1-2048_mwap"
shell_script_name = "slurm_ef21_pp_k1-64_f015-0125_prwall_mwap"
shell_script_name = "slurm_ef21_pp_k1-2_f512-9192mwap"
#shell_script_name = "slurm_ef21_bc_k_bd32-64_mwap"

single_experiment_name = "mwap"


print ("bash "+shell_script_name + ".sh\n")

#ef21-pp_nw-{0}_{1}x_{2}

with open(shell_script_name + ".sh", 'w') as f1:
    if "ef21_fg" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_fg_k_ar, ef21_fg_factor_ar, n_ar))))
        for i, (dataset, k_tk, factor, n_w) in enumerate (itertools.product (dataset_ar, ef21_fg_k_ar, ef21_fg_factor_ar, n_ar)):
            script_name = shell_script_name+"_"+"ef21_fg_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    max_it = {"w8a":int(ef21_fg_w8a_num_iter/k_tk), "a9a":int(ef21_fg_a9a_num_iter/k_tk), 
                              "mushrooms":int(ef21_fg_mushrooms_num_iter/k_tk), "phishing":int(ef21_fg_phishing_num_iter/k_tk)}
                    print (f"python3 ef21_fg.py --k {k_tk} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "ef21_plus_fg" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_plus_fg_k_ef_ar, ef21_plus_fg_k_cgd_ar, ef21_plus_fg_factor_ar, n_ar))))
        for i, (dataset, k_ef, k_cgd, factor, n_w) in enumerate (itertools.product (dataset_ar, ef21_plus_fg_k_ef_ar, ef21_plus_fg_k_cgd_ar, ef21_plus_fg_factor_ar, n_ar)):
            script_name = shell_script_name+"_"+"ef21_plus_fg_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    min_k = min (k_ef, k_cgd)
                    max_it = {"w8a":int(ef21_plus_fg_w8a_num_iter/min_k), "a9a":int(ef21_plus_fg_a9a_num_iter/min_k), 
                              "mushrooms":int(ef21_plus_fg_mushrooms_num_iter/min_k), "phishing":int(ef21_plus_fg_phishing_num_iter/min_k)}
                    print (f"python3 ef21_plus_fg.py --k_ef {k_ef} --k_cgd {k_cgd} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "ef21_pp" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_pp_k_ar, ef21_pp_factor_ar, n_ar, ef21_pp_prw_ar))))
        for i, (dataset, k_tk, factor, n_w, prw) in enumerate (itertools.product (dataset_ar, ef21_pp_k_ar, ef21_pp_factor_ar, n_ar, ef21_pp_prw_ar)):
            script_name = shell_script_name+"_"+"ef21_pp_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    if dependence_steps_k:
                        max_it = {"w8a":int(ef21_pp_w8a_num_iter/k_tk), "a9a":int(ef21_pp_a9a_num_iter/k_tk), "mushrooms":int(ef21_pp_mushrooms_num_iter/k_tk), "phishing":int(ef21_pp_phishing_num_iter/k_tk)}
                    else:
                        max_it = {"w8a":int(ef21_pp_w8a_num_iter), "a9a":int(ef21_pp_a9a_num_iter), "mushrooms":int(ef21_pp_mushrooms_num_iter), "phishing":int(ef21_pp_phishing_num_iter)}
                    print (f"python3 ef21_pp.py --k {k_tk} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w} --prw {prw}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "ef21_sgd" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_sgd_k_ar, ef21_sgd_factor_ar, n_ar, ef21_sgd_prb_ar))))
        for i, (dataset, k_tk, factor, n_w, prb) in enumerate (itertools.product (dataset_ar, ef21_sgd_k_ar, ef21_sgd_factor_ar, n_ar, ef21_sgd_prb_ar)):
            script_name = shell_script_name+"_"+"ef21_sgd_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    max_it = {"w8a":int(ef21_sgd_w8a_num_iter/k_tk), "a9a":int(ef21_sgd_a9a_num_iter/k_tk), 
                              "mushrooms":int(ef21_sgd_mushrooms_num_iter/k_tk), "phishing":int(ef21_sgd_phishing_num_iter/k_tk)}
                    print (f"python3 ef21_sgd.py --k {k_tk} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w} --prb {prb}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "ef21_page" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_page_k_ar, ef21_page_factor_ar, n_ar, ef21_page_prb_ar))))
        for i, (dataset, k_tk, factor, n_w, prb) in enumerate (itertools.product (dataset_ar, ef21_page_k_ar, ef21_page_factor_ar, n_ar, ef21_page_prb_ar)):
            script_name = shell_script_name+"_"+"ef21_page_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    max_it = {"w8a":int(ef21_page_w8a_num_iter/k_tk), "a9a":int(ef21_page_a9a_num_iter/k_tk), 
                              "mushrooms":int(ef21_page_mushrooms_num_iter/k_tk), "phishing":int(ef21_page_phishing_num_iter/k_tk)}
                    print (f"python3 ef21_page.py --k {k_tk} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w} --prb {prb}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "ef21_hb" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_hb_k_ar, ef21_hb_factor_ar, n_ar, ef21_hb_eta_ar))))
        for i, (dataset, k_tk, factor, n_w,eta) in enumerate (itertools.product (dataset_ar, ef21_hb_k_ar, ef21_hb_factor_ar, n_ar, ef21_hb_eta_ar)):
            script_name = shell_script_name+"_"+"ef21_hb_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    max_it = {"w8a":int(ef21_hb_w8a_num_iter/k_tk), "a9a":int(ef21_hb_a9a_num_iter/k_tk), 
                              "mushrooms":int(ef21_hb_mushrooms_num_iter/k_tk), "phishing":int(ef21_hb_phishing_num_iter/k_tk)}
                    print (f"python3 ef21_hb.py --k {k_tk} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w} --eta {eta}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "hb" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, hb_factor_ar, n_ar, hb_eta_ar))))
        for i, (dataset, factor, n_w,eta) in enumerate (itertools.product (dataset_ar, hb_factor_ar, n_ar, hb_eta_ar)):
            script_name = shell_script_name+"_"+"hb_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    max_it = {"w8a":int(hb_w8a_num_iter), "a9a":int(hb_a9a_num_iter), 
                              "mushrooms":int(hb_mushrooms_num_iter), "phishing":int(hb_phishing_num_iter)}
                    print (f"python3 hb.py --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w} --eta {eta}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "gd" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, gd_factor_ar, n_ar))))
        for i, (dataset, factor, n_w) in enumerate (itertools.product (dataset_ar, gd_factor_ar, n_ar)):
            script_name = shell_script_name+"_"+"gd_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    max_it = {"w8a":int(gd_w8a_num_iter), "a9a":int(gd_a9a_num_iter), 
                              "mushrooms":int(gd_mushrooms_num_iter), "phishing":int(gd_phishing_num_iter)}
                    print (f"python3 gd.py --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "ef21_bc" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_bc_k_od_ar, ef21_bc_k_bd_ar, ef21_bc_factor_ar, n_ar))))
        for i, (dataset, k_od, k_bd, factor, n_w) in enumerate (itertools.product (dataset_ar, ef21_bc_k_od_ar, ef21_bc_k_bd_ar, ef21_bc_factor_ar, n_ar)):
            script_name = shell_script_name+"_"+"ef21_bc_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    min_k = min (k_od, k_bd)
                    max_it = {"w8a":int(ef21_bc_w8a_num_iter), "a9a":int(ef21_bc_a9a_num_iter), 
                              "mushrooms":int(ef21_bc_mushrooms_num_iter), "phishing":int(ef21_bc_phishing_num_iter)}
                    print (f"python3 ef21_bc.py --k_od {k_od} --k_bd {k_bd} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)

if print_global_script:    
    f = open(shell_script_name + ".sh", 'r')
    print(f.read())
    f.close()


bash slurm_ef21_pp_k1-2_f512-9192mwap.sh

Number of experiments:  80
#!/bin/bash
python3 ef21_pp.py --k 2 --dataset mushrooms --max_it 30000 --tol 1e-07 --factor 512.0 --num_workers 100 --prw 0.95

#!/bin/bash
python3 ef21_pp.py --k 2 --dataset mushrooms --max_it 30000 --tol 1e-07 --factor 512.0 --num_workers 100 --prw 0.5

#!/bin/bash
python3 ef21_pp.py --k 2 --dataset mushrooms --max_it 30000 --tol 1e-07 --factor 512.0 --num_workers 100 --prw 0.25

#!/bin/bash
python3 ef21_pp.py --k 2 --dataset mushrooms --max_it 30000 --tol 1e-07 --factor 512.0 --num_workers 100 --prw 0.125

#!/bin/bash
python3 ef21_pp.py --k 2 --dataset mushrooms --max_it 30000 --tol 1e-07 --factor 1024.0 --num_workers 100 --prw 0.95

#!/bin/bash
python3 ef21_pp.py --k 2 --dataset mushrooms --max_it 30000 --tol 1e-07 --factor 1024.0 --num_workers 100 --prw 0.5

#!/bin/bash
python3 ef21_pp.py --k 2 --dataset mushrooms --max_it 30000 --tol 1e-07 --factor 1024.0 --num_workers 100 --prw 0.25

#!/bin/bash
python3 ef21_p

## amadorja

In [9]:
myrepr = lambda x: repr(round(x, 4)).replace('.',',') if isinstance(x, float) else repr(x)

def read_file (shell_script_name, print_each_script):
    if print_each_script:
        f = open(shell_script_name, 'r')
        print(f.read())
        f.close()

project_path = os.getcwd() + "/"

dataset_ar = ["mushrooms","w8a","a9a","phishing"]
dataset_ar = ["realsim"]
#dataset_ar = ["mushrooms","w8a","a9a","phishing","realsim"]
#dataset_ar = ["mushrooms","a9a","phishing"]
#dataset_ar = ["mushrooms"]
dataset_ar = ["w8a"]
dataset_ar = ["a9a"]

exps = ["ef21_fg"]
#exps = ["ef21_mnt"]
#exps = ["ef21_pp"]
#exps = ["ef21_sgd"]
#exps = ["sgd"]
#exps = ["ef21_hb"]
#exps = ["ef21_plus_fg"]
#exps = ["ef21_page"]
#exps = ["hb"]
#exps = ["gd"]
#exps = ["ef21_bc"]

tol = 1e-7

##EF21 with full gradients##
ef21_fg_mushrooms_num_iter = 2355962
ef21_fg_w8a_num_iter = 3000
ef21_fg_a9a_num_iter = 56249
ef21_fg_phishing_num_iter = 4549
ef21_fg_realsim_num_iter = 15000
ef21_fg_k_ar = [3,30,75,150]

ef21_fg_k_ar = [1,3,30,60]

ef21_fg_factor_ar = [1,2,4,8,16,32,64,128,256,512,1024,2048,4096]

## EF21 with HB ##
ef21_hb_mushrooms_num_iter = 100_000
ef21_hb_w8a_num_iter = 100000
ef21_hb_a9a_num_iter = 10000
ef21_hb_phishing_num_iter = 10000
ef21_hb_k_ar = [1,2,4,8,16]
#ef21_hb_k_ar = [1]
ef21_hb_factor_ar = [1,2,4,8,16,32,64,128,256,512,1024,2048]
#ef21_hb_factor_ar = [64,128,256,512,1024,2048,4096]
ef21_hb_eta_ar = np.array([0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,0.99]) # final set for all 
#ef21_hb_eta_ar = np.sort( np.concatenate ((np.linspace(start=0.01, stop=0.99,endpoint=True,num=20),np.array([0.9,0.95])))) #for a9a

## EF21 with PAGE ##
ef21_page_mushrooms_num_iter = 50_000
ef21_page_w8a_num_iter = 10_000
ef21_page_a9a_num_iter = 10_000
ef21_page_phishing_num_iter = 10_000
ef21_page_k_ar = [1,2,4,8,16]
ef21_page_factor_ar = [0.25,0.5,1,2,4,8,16,32,64,128,256,512,1024,2048]
ef21_page_prb_ar = [0.95,0.5,0.25,0.125,0.065,0.03,0.015,0.007]
ef21_page_prb_ar = [0.0035]

## EF21 with PP ##
ef21_pp_mushrooms_num_iter = 10_000
ef21_pp_w8a_num_iter = 2_000
ef21_pp_a9a_num_iter = 2_000
ef21_pp_phishing_num_iter = 10_000
ef21_pp_realsim_num_iter = 6_000
ef21_pp_k_ar = [210]
ef21_pp_factor_ar = np.array([8,16,32,64,128,256,512,1024],dtype=float)
ef21_pp_prw_ar = [0.5,0.25,0.125,0.065]

## EF21 with SGD ##
ef21_fg_mushrooms_best = 2300
ef21_fg_w8a_best = 1500
ef21_fg_a9a_best = 1000
ef21_fg_phishing_best = 1000

ef21_sgd_mushrooms_num_iter = int(ef21_fg_mushrooms_best/0.015)
ef21_sgd_w8a_num_iter = int(ef21_fg_w8a_best/0.015)
ef21_sgd_a9a_num_iter = int(ef21_fg_a9a_best/0.015)
ef21_sgd_phishing_num_iter = int(ef21_fg_phishing_best/0.015)
ef21_sgd_realsim_num_iter = int(500/0.015)

ef21_sgd_prb_ar = [0.25, 0.125]
ef21_sgd_factor_ar = np.array([4,8,16,32,64,128,256,512,1024,2048,4096], dtype=float)
ef21_sgd_k_ar = [210]

## HB ##
hb_mushrooms_num_iter = 1_000
hb_w8a_num_iter = 10_000
hb_a9a_num_iter = 5_000
hb_phishing_num_iter = 5_000

hb_factor_ar = [0.125,0.25,0.5, 1,2,4,8,16,32,64,128,256,512,1024,2048]
hb_factor_ar = [4096, 8192, 18382, 36768]
#hb_factor_ar = [0.25]
hb_eta_ar = np.array([0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,0.99]) # final set for all 

## GD ##
gd_mushrooms_num_iter = 1_000
gd_w8a_num_iter = 1_000
gd_a9a_num_iter = 1_000
gd_phishing_num_iter = 1_000

gd_factor_ar = [1,2,4,8,16,32,64,128,256,512,1024,2048]
gd_factor_ar = [0.125,0.25,0.5]

##EF21 with BC##
ef21_bc_mushrooms_num_iter = 5_000
ef21_bc_w8a_num_iter = 3_000
ef21_bc_a9a_num_iter = 1_000
ef21_bc_phishing_num_iter = 1000

ef21_bc_k_od_ar = [1,2,4,8,16] #ef21 top-k
ef21_bc_k_bd_ar = [1,2,4,8,16]
ef21_bc_k_od_ar = [32,64]
ef21_bc_k_bd_ar = [32,64]
ef21_bc_factor_ar = [0.125,0.25,0.5, 1,2,4,8,16,32,64,128,256,512,1024,2048]
#ef21_bc_factor_ar = [0.125]

## SGD ##
#sgd_k_ar = {'mushrooms': [2], 'w8a': [2], 'a9a': [2], 'phishing': [1], 'realsim': [210]}[dataset]
sgd_mushrooms_best = 2300
sgd_w8a_best = 1500
sgd_a9a_best = 1000
sgd_phishing_best = 1000
sgd_realsim_best = 2300

sgd_mushrooms_num_iter = int(sgd_mushrooms_best/0.015)
sgd_w8a_num_iter = int(sgd_w8a_best/0.015)
sgd_a9a_num_iter = int(sgd_a9a_best/0.015)
sgd_phishing_num_iter = int(sgd_phishing_best/0.015)
sgd_realsim_num_iter = int(sgd_realsim_best/0.015)

#test

sgd_mushrooms_num_iter = int(500/0.015)
sgd_w8a_num_iter = int(250/0.015)
sgd_a9a_num_iter = int(70/0.015)
sgd_phishing_num_iter = int(70/0.015)

sgd_realsim_num_iter = int(500/0.015)

sgd_prb_ar = [0.25, 0.125, 0.015]
sgd_factor_ar = np.array([4,8,16,32,64,128,256,512,1024,2048,4096],dtype=float)

if "ef21_pp" in exps:
    n_ar = [100]
else:
    n_ar = [20]

n_ar = [100]
time = 10
cpus_per_task = 1
memmory = 8

dependence_steps_k = 0
print_each_script = 1
print_global_script = 1

shell_script_name = "slurm_ef21-fg-100_aw"
single_experiment_name = "aw"


print ("bash "+shell_script_name + ".sh\n")

#ef21-pp_nw-{0}_{1}x_{2}

with open(shell_script_name + ".sh", 'w') as f1:
    if "ef21_fg" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_fg_k_ar, ef21_fg_factor_ar, n_ar))))
        for i, (dataset, k_tk, factor, n_w) in enumerate (itertools.product (dataset_ar, ef21_fg_k_ar, ef21_fg_factor_ar, n_ar)):
            script_name = shell_script_name+"_"+"ef21_fg_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    if dependence_steps_k:
                        max_it = {"w8a":int(ef21_fg_w8a_num_iter/k_tk), "a9a":int(ef21_fg_a9a_num_iter/k_tk), 
                              "mushrooms":int(ef21_fg_mushrooms_num_iter/k_tk), "phishing":int(ef21_fg_phishing_num_iter/k_tk), "realsim":int(ef21_fg_realsim_num_iter/k_tk)}
                    else:
                        max_it = {"w8a":int(ef21_fg_w8a_num_iter), "a9a":int(ef21_fg_a9a_num_iter), 
                              "mushrooms":int(ef21_fg_mushrooms_num_iter), "phishing":int(ef21_fg_phishing_num_iter), "realsim":int(ef21_fg_realsim_num_iter)}
                    print (f"python3 ef21_fg.py --k {k_tk} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w}")
            memmory = {'mushrooms': 8, 'w8a': 8, 'a9a': 8, 'phishing': 8, 'realsim': 128 }[dataset]
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "ef21_plus_fg" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_plus_fg_k_ef_ar, ef21_plus_fg_k_cgd_ar, ef21_plus_fg_factor_ar, n_ar))))
        for i, (dataset, k_ef, k_cgd, factor, n_w) in enumerate (itertools.product (dataset_ar, ef21_plus_fg_k_ef_ar, ef21_plus_fg_k_cgd_ar, ef21_plus_fg_factor_ar, n_ar)):
            script_name = shell_script_name+"_"+"ef21_plus_fg_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    min_k = min (k_ef, k_cgd)
                    max_it = {"w8a":int(ef21_plus_fg_w8a_num_iter/min_k), "a9a":int(ef21_plus_fg_a9a_num_iter/min_k), 
                              "mushrooms":int(ef21_plus_fg_mushrooms_num_iter/min_k), "phishing":int(ef21_plus_fg_phishing_num_iter/min_k)}
                    print (f"python3 ef21_plus_fg.py --k_ef {k_ef} --k_cgd {k_cgd} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "ef21_pp" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_pp_k_ar, ef21_pp_factor_ar, n_ar, ef21_pp_prw_ar))))
        for i, (dataset, k_tk, factor, n_w, prw) in enumerate (itertools.product (dataset_ar, ef21_pp_k_ar, ef21_pp_factor_ar, n_ar, ef21_pp_prw_ar)):
            script_name = shell_script_name+"_"+"ef21_pp_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    if dependence_steps_k:
                        max_it = {"w8a":int(ef21_pp_w8a_num_iter/k_tk), "a9a":int(ef21_pp_a9a_num_iter/k_tk), "mushrooms":int(ef21_pp_mushrooms_num_iter/k_tk), "phishing":int(ef21_pp_phishing_num_iter/k_tk), "realsim":int(ef21_pp_realsim_num_iter/k_tk)}
                    else:
                        max_it = {"w8a":int(ef21_pp_w8a_num_iter), "a9a":int(ef21_pp_a9a_num_iter), "mushrooms":int(ef21_pp_mushrooms_num_iter), "phishing":int(ef21_pp_phishing_num_iter), "realsim":int(ef21_pp_realsim_num_iter)}
                    print (f"python3 ef21_pp.py --k {k_tk} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w} --prw {prw}")
            memmory = {'mushrooms': 8, 'w8a': 8, 'a9a': 8, 'phishing': 8, 'realsim': 128 }[dataset]
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "ef21_sgd" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_sgd_k_ar, ef21_sgd_factor_ar, n_ar, ef21_sgd_prb_ar))))
        for i, (dataset, k_tk, factor, n_w, prb) in enumerate (itertools.product (dataset_ar, ef21_sgd_k_ar, ef21_sgd_factor_ar, n_ar, ef21_sgd_prb_ar)):
            script_name = shell_script_name+"_"+"ef21_sgd_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    if dependence_steps_k:
                        max_it = {"w8a":int(ef21_sgd_w8a_num_iter/k_tk), "a9a":int(ef21_sgd_a9a_num_iter/k_tk),"mushrooms":int(ef21_sgd_mushrooms_num_iter/k_tk), "phishing":int(ef21_sgd_phishing_num_iter/k_tk), "realsim":int(ef21_sgd_realsim_num_iter/k_tk)}
                    else:
                        max_it = {"w8a":int(ef21_sgd_w8a_num_iter), "a9a":int(ef21_sgd_a9a_num_iter),"mushrooms":int(ef21_sgd_mushrooms_num_iter), "phishing":int(ef21_sgd_phishing_num_iter), "realsim":int(ef21_sgd_realsim_num_iter)}
                    print (f"python3 ef21_sgd.py --k {k_tk} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w} --prb {prb}")
            memmory = {'mushrooms': 8, 'w8a': 8, 'a9a': 8, 'phishing': 8, 'realsim': 128 }[dataset]
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    
    if "sgd" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, sgd_factor_ar, n_ar, sgd_prb_ar))))
        for i, (dataset, factor, n_w, prb) in enumerate (itertools.product (dataset_ar, sgd_factor_ar, n_ar, sgd_prb_ar)):
            script_name = shell_script_name+"_"+"sgd_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    max_it = {"w8a":int(sgd_w8a_num_iter), "a9a":int(sgd_a9a_num_iter),"mushrooms":int(sgd_mushrooms_num_iter), "phishing":int(sgd_phishing_num_iter), "realsim":int(sgd_realsim_num_iter)}
                    print (f"python3 sgd.py --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w} --prb {prb}")
            memmory = {'mushrooms': 8, 'w8a': 8, 'a9a': 8, 'phishing': 8, 'realsim': 128 }[dataset]
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
            
    if "ef21_page" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_page_k_ar, ef21_page_factor_ar, n_ar, ef21_page_prb_ar))))
        for i, (dataset, k_tk, factor, n_w, prb) in enumerate (itertools.product (dataset_ar, ef21_page_k_ar, ef21_page_factor_ar, n_ar, ef21_page_prb_ar)):
            script_name = shell_script_name+"_"+"ef21_page_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    if dependence_steps_k:
                        max_it = {"w8a":int(ef21_page_w8a_num_iter/k_tk), "a9a":int(ef21_page_a9a_num_iter/k_tk),"mushrooms":int(ef21_page_mushrooms_num_iter/k_tk), "phishing":int(ef21_page_phishing_num_iter/k_tk)}
                    else:
                        max_it = {"w8a":int(ef21_page_w8a_num_iter), "a9a":int(ef21_page_a9a_num_iter),"mushrooms":int(ef21_page_mushrooms_num_iter), "phishing":int(ef21_page_phishing_num_iter)}
                    print (f"python3 ef21_page.py --k {k_tk} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w} --prb {prb}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "ef21_hb" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_hb_k_ar, ef21_hb_factor_ar, n_ar, ef21_hb_eta_ar))))
        for i, (dataset, k_tk, factor, n_w,eta) in enumerate (itertools.product (dataset_ar, ef21_hb_k_ar, ef21_hb_factor_ar, n_ar, ef21_hb_eta_ar)):
            script_name = shell_script_name+"_"+"ef21_hb_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    max_it = {"w8a":int(ef21_hb_w8a_num_iter/k_tk), "a9a":int(ef21_hb_a9a_num_iter/k_tk), 
                              "mushrooms":int(ef21_hb_mushrooms_num_iter/k_tk), "phishing":int(ef21_hb_phishing_num_iter/k_tk)}
                    print (f"python3 ef21_hb.py --k {k_tk} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w} --eta {eta}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "hb" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, hb_factor_ar, n_ar, hb_eta_ar))))
        for i, (dataset, factor, n_w,eta) in enumerate (itertools.product (dataset_ar, hb_factor_ar, n_ar, hb_eta_ar)):
            script_name = shell_script_name+"_"+"hb_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    max_it = {"w8a":int(hb_w8a_num_iter), "a9a":int(hb_a9a_num_iter), 
                              "mushrooms":int(hb_mushrooms_num_iter), "phishing":int(hb_phishing_num_iter)}
                    print (f"python3 hb.py --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w} --eta {eta}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "gd" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, gd_factor_ar, n_ar))))
        for i, (dataset, factor, n_w) in enumerate (itertools.product (dataset_ar, gd_factor_ar, n_ar)):
            script_name = shell_script_name+"_"+"gd_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    max_it = {"w8a":int(gd_w8a_num_iter), "a9a":int(gd_a9a_num_iter), 
                              "mushrooms":int(gd_mushrooms_num_iter), "phishing":int(gd_phishing_num_iter)}
                    print (f"python3 gd.py --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)
    if "ef21_bc" in exps:
        print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_bc_k_od_ar, ef21_bc_k_bd_ar, ef21_bc_factor_ar, n_ar))))
        for i, (dataset, k_od, k_bd, factor, n_w) in enumerate (itertools.product (dataset_ar, ef21_bc_k_od_ar, ef21_bc_k_bd_ar, ef21_bc_factor_ar, n_ar)):
            script_name = shell_script_name+"_"+"ef21_bc_"+single_experiment_name +f"_{i}" + ".sh"
            with open(script_name, 'w') as f:
                with redirect_stdout(f):
                    print ("#!/bin/bash")
                    min_k = min (k_od, k_bd)
                    max_it = {"w8a":int(ef21_bc_w8a_num_iter/min_k), "a9a":int(ef21_bc_a9a_num_iter/min_k), 
                              "mushrooms":int(ef21_bc_mushrooms_num_iter/min_k), "phishing":int(ef21_bc_phishing_num_iter/min_k)}
                    print (f"python3 ef21_bc.py --k_od {k_od} --k_bd {k_bd} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w}")
            f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
            read_file (script_name, print_each_script)

if print_global_script:    
    f = open(shell_script_name + ".sh", 'r')
    print(f.read())
    f.close()


bash slurm_ef21-fg-100_a.sh

Number of experiments:  52
#!/bin/bash
python3 ef21_fg.py --k 1 --dataset a9a --max_it 56249 --tol 1e-07 --factor 1 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 1 --dataset a9a --max_it 56249 --tol 1e-07 --factor 2 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 1 --dataset a9a --max_it 56249 --tol 1e-07 --factor 4 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 1 --dataset a9a --max_it 56249 --tol 1e-07 --factor 8 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 1 --dataset a9a --max_it 56249 --tol 1e-07 --factor 16 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 1 --dataset a9a --max_it 56249 --tol 1e-07 --factor 32 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 1 --dataset a9a --max_it 56249 --tol 1e-07 --factor 64 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 1 --dataset a9a --max_it 56249 --tol 1e-07 --factor 128 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 1 --dataset a9a --max_it 56249 --tol 1e-

# ef21-fg

In [7]:
myrepr = lambda x: repr(round(x, 4)).replace('.',',') if isinstance(x, float) else repr(x)

def read_file (shell_script_name, print_each_script):
    if print_each_script:
        f = open(shell_script_name, 'r')
        print(f.read())
        f.close()

project_path = os.getcwd() + "/"

dataset_ar = ["w8a","a9a"]

exps = ["ef21_fg"]

tol = 1e-7

##EF21 with full gradients##
ef21_fg_mushrooms_num_iter = 2000
ef21_fg_w8a_num_iter = 2000
ef21_fg_a9a_num_iter = 2000
ef21_fg_phishing_num_iter = 4549
ef21_fg_realsim_num_iter = 15000

ef21_fg_k_ar_dict = {"w8a":[3,30,75,150], "a9a":[1,3,30,60]}

ef21_fg_factor_ar_dict  = {"w8a":{3:64, 30:32, 75:32, 150:16}, "a9a":{1:128, 3:64, 30:16, 60:8}}

n_ar = [100]
time = 5
cpus_per_task = 1
memmory = 16

dependence_steps_k = 0
print_each_script = 1
print_global_script = 1

shell_script_name = "slurm_ef21-fg-100_aw"
single_experiment_name = "aw"

print ("bash "+shell_script_name + ".sh\n")

#ef21-pp_nw-{0}_{1}x_{2}

with open(shell_script_name + ".sh", 'w') as f1:
    if "ef21_fg" in exps:
        #print ("Number of experiments: ", len(list(itertools.product (dataset_ar, ef21_fg_k_ar, ef21_fg_factor_ar, n_ar))))
        for dataset in dataset_ar:
            ef21_fg_k_ar = ef21_fg_k_ar_dict[dataset]
            for i, (k_tk, n_w) in enumerate (itertools.product (ef21_fg_k_ar, n_ar)):
                factor = ef21_fg_factor_ar_dict[dataset][k_tk]
                script_name = shell_script_name+"_"+"ef21_fg_"+single_experiment_name +f"_{i}" + ".sh"
                with open(script_name, 'w') as f:
                    with redirect_stdout(f):
                        print ("#!/bin/bash")
                        if dependence_steps_k:
                            max_it = {"w8a":int(ef21_fg_w8a_num_iter/k_tk), "a9a":int(ef21_fg_a9a_num_iter/k_tk), 
                                  "mushrooms":int(ef21_fg_mushrooms_num_iter/k_tk), "phishing":int(ef21_fg_phishing_num_iter/k_tk), "realsim":int(ef21_fg_realsim_num_iter/k_tk)}
                        else:
                            max_it = {"w8a":int(ef21_fg_w8a_num_iter), "a9a":int(ef21_fg_a9a_num_iter), 
                                  "mushrooms":int(ef21_fg_mushrooms_num_iter), "phishing":int(ef21_fg_phishing_num_iter), "realsim":int(ef21_fg_realsim_num_iter)}
                        print (f"python3 ef21_fg.py --k {k_tk} --dataset {dataset} --max_it {max_it[dataset]} --tol {tol} --factor {factor} --num_workers {n_w}")
                memmory = {'mushrooms': 8, 'w8a': 8, 'a9a': 8, 'phishing': 8, 'realsim': 128 }[dataset]
                f1.write ("sbatch --cpus-per-task={3} --mem={4}GB --time={2}:00:00 {0}{1}\n".format(project_path, script_name, time, cpus_per_task, memmory))
                read_file (script_name, print_each_script)
    

if print_global_script:    
    f = open(shell_script_name + ".sh", 'r')
    print(f.read())
    f.close()


bash slurm_ef21-fg-100_aw.sh

#!/bin/bash
python3 ef21_fg.py --k 3 --dataset w8a --max_it 2000 --tol 1e-07 --factor 64 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 30 --dataset w8a --max_it 2000 --tol 1e-07 --factor 32 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 75 --dataset w8a --max_it 2000 --tol 1e-07 --factor 32 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 150 --dataset w8a --max_it 2000 --tol 1e-07 --factor 16 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 1 --dataset a9a --max_it 2000 --tol 1e-07 --factor 128 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 3 --dataset a9a --max_it 2000 --tol 1e-07 --factor 64 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 30 --dataset a9a --max_it 2000 --tol 1e-07 --factor 16 --num_workers 100

#!/bin/bash
python3 ef21_fg.py --k 60 --dataset a9a --max_it 2000 --tol 1e-07 --factor 8 --num_workers 100

sbatch --cpus-per-task=1 --mem=8GB --time=5:00:00 /home/fatkhui/GN_most_recent_exps/slurm_ef21-fg-10